In [2]:
from tensorflow import keras
import tensorflow as tf
import keras
import pandas as pd 


# Load Data

In [3]:
raw_data = pd.read_csv('airline-passengers.txt')
passengers = raw_data.iloc[1:,1]
print(len(passengers))


143


# PreProcessing

In [4]:
train_percent = 0.67
test_percent = 0.18
val_percent = 0.15
num_train_sample = int(train_percent * len(passengers)) # number of train samples
num_val_sample = int(val_percent * len(passengers)) # number of validation samples
num_test_sample = len(passengers) - num_train_sample - num_val_sample # number of test samples
print('Test: ',num_test_sample)
print('Train: ',num_train_sample)
print('Validation: ',num_val_sample)
# Normalization
mean = passengers[:num_train_sample].mean(axis=0)
passengers -= mean
std = passengers[:num_train_sample].std(axis=0)
passengers /= std

Test:  27
Train:  95
Validation:  21


# Labeling

In [5]:
sampling_rate = 1
sequence_length = 4
batch_size=8
delay = sampling_rate*(sequence_length -1)

train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    passengers[:-delay],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    targets = passengers[delay:] ,
    batch_size = batch_size,
    shuffle = True,
    start_index = 0,
    end_index = num_train_sample
    )

val_dataset = tf.keras.utils.timeseries_dataset_from_array(
    passengers[:-delay],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    targets = passengers[delay:] ,
    batch_size = batch_size,
    shuffle = True,
    start_index = num_train_sample,
    end_index = num_train_sample + num_val_sample
    )

test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    passengers[:-delay],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    targets = passengers[delay:] ,
    batch_size = batch_size,
    shuffle = True,
    start_index= num_train_sample + num_val_sample
    )

#print(lables)

# Define Network and Train

In [6]:
#inputs = keras.Input(shape=(sequence_length, passengers.shape[-1]))
'''model = tf.keras.Sequential([
    tf.keras.layers.LSTM(16)( shape=(sequence_length, passengers.shape[-1])),
    tf.keras.layers.Dense(1)]
    shape=(sequence_length, passengers.shape[-1]
)'''
print((sequence_length, passengers.shape[-1]))
inputs = keras.Input(shape=(4, 143))
print(inputs)
x = tf.keras.layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
#model.summary()



(4, 143)
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 143), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [7]:
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=val_dataset)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\Maryam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Maryam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Maryam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Maryam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Maryam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\